In [2]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [3]:
import os

adult_df = pd.read_csv(os.path.join("..", "data", "adult.csv"))

adult_df.replace("?", pd.NA, inplace=True)
adult_df = adult_df.dropna()

salary_over_50k = adult_df["salary K$"] > 50
adults_over_50k = adult_df[adult_df["salary"] == ">50K"]
salary_below_50k = adult_df["salary K$"] <= 50
adults_below_50k = adult_df[adult_df["salary"] == "<=50K"]

adults_df = pd.merge(adults_over_50k, adults_below_50k, how="outer")
adults_df.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


# Task 1
Print the count of men and women in the dataset.

In [4]:
male_amount = (adults_df["sex"] == "Male").sum()
female_amount = (adults_df["sex"] == "Female").sum()

male_amount

20380

# Task 2
Find the average age of men in dataset

In [13]:
average_men_age = adults_df[adults_df["sex"] == "Male"]["age"].mean()

average_men_age

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [14]:
amount_of_poles = (adults_df["native-country"] == "Poland").sum()
percentage = (amount_of_poles / len(adults_df)) * 100

percentage

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [15]:
avg_age_over = adults_over_50k["age"].mean()
avg_age_below = adults_below_50k["age"].mean()

std_age_over = adults_over_50k["age"].std()
std_age_below = adults_below_50k["age"].std()

std_age_over

10.269632835673852

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [16]:
degrees = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]
hustlers = adults_over_50k[~adults_over_50k["education"].isin(degrees)]

hustlers

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
7,7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307
10,10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K,116
55,55,43,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,>50K,341
67,67,53,Private,HS-grad,Married-civ-spouse,Adm-clerical,Wife,White,Female,40,United-States,>50K,225
68,68,49,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,>50K,194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32462,32462,48,Self-emp-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,50,United-States,>50K,343
32518,32518,57,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,116
32519,32519,46,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,48,United-States,>50K,239
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [17]:
education_age_stats = adults_df.groupby("education")["age"].describe()

education_age_stats

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [19]:
men = adults_df["sex"] == "Male"
married = adults_df["marital-status"].str.startswith("Married")

married_men = adults_df[men & married]
non_married_men = adults_df[men & ~married]

avg_married_salary = married_men["salary K$"].describe()
avg_non_married_salary = non_married_men["salary K$"].describe()

avg_married_salary

count    12775.000000
mean       107.494560
std        102.475154
min         15.000000
25%         30.000000
50%         46.000000
75%        184.000000
max        349.000000
Name: salary K$, dtype: float64

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [21]:
hour_groups = adults_df.groupby("hours-per-week").size().reset_index(name="count")

max_hour_amount = hour_groups.loc[hour_groups["hours-per-week"].idxmax()]
max_hour_group = hour_groups.loc[hour_groups["count"].idxmax()]

max_hour_group

hours-per-week       40
count             14251
Name: 39, dtype: int64

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [12]:
adults_encoded = pd.get_dummies(adults_df)

def get_adults_encoded_mask(regex_filter: str) -> pd.DataFrame:
    return adults_encoded.filter(regex=regex_filter)

def get_concat_salary_mask(columns: pd.Series | pd.DataFrame) -> pd.DataFrame:
    return pd.concat([columns, adults_encoded["salary K$"]], axis=1)

workclass_columns = get_adults_encoded_mask("^workclass_")
education_columns = get_adults_encoded_mask("^education_")

workclass_salary = get_concat_salary_mask(workclass_columns)
education_salary = get_concat_salary_mask(education_columns)

age_salary = get_concat_salary_mask(adults_encoded["age"])

age_salary.corr()

,age,salary K$
age,1.000000,0.208203
salary K$,0.208203,1.000000
